In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 

import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s4e7/sample_submission.csv
/kaggle/input/playground-series-s4e7/train.csv
/kaggle/input/playground-series-s4e7/test.csv


In [3]:
train = pd.read_csv("/kaggle/input/playground-series-s4e7/train.csv")

# ***EDA***

In [4]:
train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,0,Male,21,1,35.0,0,1-2 Year,Yes,65101.0,124.0,187,0
1,1,Male,43,1,28.0,0,> 2 Years,Yes,58911.0,26.0,288,1
2,2,Female,25,1,14.0,1,< 1 Year,No,38043.0,152.0,254,0
3,3,Female,35,1,1.0,0,1-2 Year,Yes,2630.0,156.0,76,0
4,4,Female,36,1,15.0,1,1-2 Year,No,31951.0,152.0,294,0


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11504798 entries, 0 to 11504797
Data columns (total 12 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    int64  
 1   Gender                object 
 2   Age                   int64  
 3   Driving_License       int64  
 4   Region_Code           float64
 5   Previously_Insured    int64  
 6   Vehicle_Age           object 
 7   Vehicle_Damage        object 
 8   Annual_Premium        float64
 9   Policy_Sales_Channel  float64
 10  Vintage               int64  
 11  Response              int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 1.0+ GB


In [10]:
quant = train.select_dtypes(include=['int64', 'float64'])

In [11]:
quant.corr()

,id,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,Response
id,1.000000,-0.000088,-0.000045,0.000147,0.000217,-0.000205,0.000550,-0.000182,0.000018
Age,-0.000088,1.000000,-0.078519,0.037041,-0.276248,0.056327,-0.591443,-0.013293,0.122134
Driving_License,-0.000045,-0.078519,1.000000,-0.001329,0.013733,-0.007300,0.042941,-0.000697,0.009197
Region_Code,0.000147,0.037041,-0.001329,1.000000,-0.022367,-0.001741,-0.037606,-0.005537,0.012816
Previously_Insured,0.000217,-0.276248,0.013733,-0.022367,1.000000,0.007665,0.236838,0.019431,-0.345930
Annual_Premium,-0.000205,0.056327,-0.007300,-0.001741,0.007665,1.000000,-0.128559,0.003284,0.032261
Policy_Sales_Channel,0.000550,-0.591443,0.042941,-0.037606,0.236838,-0.128559,1.000000,0.008999,-0.152733
Vintage,-0.000182,-0.013293,-0.000697,-0.005537,0.019431,0.003284,0.008999,1.000000,-0.015177
Response,0.000018,0.122134,0.009197,0.012816,-0.345930,0.032261,-0.152733,-0.015177,1.000000


In [21]:
cat = train[['Vehicle_Age', 'Vehicle_Damage', 'Gender']]

train = pd.get_dummies(train, drop_first=True)

In [24]:
train = train.drop(['Driving_License', 'id'], axis=1)